In [1]:
#@title Check if installation was successful

#from google.colab import files

import distutils.util
import os
import subprocess
if subprocess.run('nvidia-smi').returncode:
  raise RuntimeError(
      'Cannot communicate with GPU. '
      'Make sure you are using a GPU Colab runtime. '
      'Go to the Runtime menu and select Choose runtime type.')

# Add an ICD config so that glvnd can pick up the Nvidia EGL driver.
# This is usually installed as part of an Nvidia driver package, but the Colab
# kernel doesn't install its driver via APT, and as a result the ICD is missing.
# (https://github.com/NVIDIA/libglvnd/blob/master/src/EGL/icd_enumeration.md)
NVIDIA_ICD_CONFIG_PATH = '/usr/share/glvnd/egl_vendor.d/10_nvidia.json'
if not os.path.exists(NVIDIA_ICD_CONFIG_PATH):
  with open(NVIDIA_ICD_CONFIG_PATH, 'w') as f:
    f.write("""{
    "file_format_version" : "1.0.0",
    "ICD" : {
        "library_path" : "libEGL_nvidia.so.0"
    }
}
""")

# Configure MuJoCo to use the EGL rendering backend (requires GPU)
print('Setting environment variable to use GPU rendering:')
%env MUJOCO_GL=egl

try:
  print('Checking that the installation succeeded:')
  import mujoco
  mujoco.MjModel.from_xml_string('<mujoco/>')
except Exception as e:
  raise e from RuntimeError(
      'Something went wrong during installation. Check the shell output above '
      'for more information.\n'
      'If using a hosted Colab runtime, make sure you enable GPU acceleration '
      'by going to the Runtime menu and selecting "Choose runtime type".')

print('Installation successful.')

#@title Import packages for plotting and creating graphics
import time
import itertools
import numpy as np
from typing import Callable, NamedTuple, Optional, Union, List

# Graphics and plotting.
print('Installing mediapy:')
!command -v ffmpeg >/dev/null || (apt update && apt install -y ffmpeg)
!pip install -q mediapy
import mediapy as media
import matplotlib.pyplot as plt

# More legible printing from numpy.
np.set_printoptions(precision=3, suppress=True, linewidth=100)

Thu Nov  2 06:32:20 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.120                Driver Version: 537.58       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA TITAN RTX               On  | 00000000:01:00.0  On |                  N/A |
| 42%   54C    P8              15W / 280W |   3332MiB / 24576MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

/tmp/ipykernel_22932/3038417431.py:5: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  import distutils.util


In [7]:
import random
import gymnasium as gym
from stable_baselines3 import SAC
from wandb.integration.sb3 import WandbCallback
from stable_baselines3.common.vec_env import DummyVecEnv
import wandb
import cv2 

env = gym.make('Ant', render_mode='rgb_array')
env.reset()
n_frames = 120
height = 480
width = 480
#import mujoco
model = env.model
data = env.data

mujoco.mj_step(model, data)
renderer = mujoco.Renderer(model, height, width)
mujoco.mj_resetDataKeyframe(model, data,0)


num_episodes = 1
for epi in range(num_episodes):  
  frames = []
  obs = env.reset()
  total_reward = 0
  
  if len(obs) == 2:
    obs = obs[0]
  for i in range(n_frames):
    while data.time < i/30.0:   
      # sample random action
      action = env.action_space.sample()
      # ob, reward, terminated, False, {}
      obs, reward, terminated, _, _ = env.step(action)
      #mujoco.mj_step(model, data)
    
    agent_position = data.qpos[0]  # Hopper의 경우 첫 번째 값이 위치일 수 있습니다.

    renderer.update_scene(data, "track") # track or y
    frame = renderer.render()
    # frame is (480, 480, 3) numpy array
    # convert to cv2 format
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    # draw total reward
    total_reward += reward
    cv2.putText(frame, f"total reward: {total_reward:.1f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    # draw reward
    cv2.putText(frame, f"reward: {reward:.1f}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    # convert back to RGB format, numpy array
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frames.append(frame)
  media.show_video(frames, fps=30)
  media.write_video(f"video_{epi}.mp4", frames, fps=30)


This browser does not support the video tag.
